In [7]:
import pandas as pd

# Sample inventory data
data = {
    'store_id': [1, 1, 1, 2, 2, 2],
    'article_id': [101, 101, 102, 101, 102, 102],
    'date': ['2023-10-01', '2023-10-03', '2023-10-01', '2023-10-01', '2023-10-01', '2023-10-02'],
    'inventory': [10, 15, 5, 20, 25, 30]
}

# Create DataFrame
df = pd.DataFrame(data)

# Convert date column to datetime
df['date'] = pd.to_datetime(df['date'])

# Sum the elements in the inventory column
total_inventory = df['inventory'].sum()
print(f"Total Inventory: {total_inventory}")

# Initialize an empty DataFrame to store missing records
missing_records = pd.DataFrame()

# Group by article_id and store_id
for (article_id, store_id), group in df.groupby(['article_id', 'store_id']):
    # Create a complete date range for the period of interest for each article and store
    date_range = pd.date_range(start=group['date'].min(), end=group['date'].max())
    
    # Create a MultiIndex with all combinations of store_id, article_id, and date
    idx = pd.MultiIndex.from_product([[store_id], [article_id], date_range], names=['store_id', 'article_id', 'date'])
    
    # Reindex the DataFrame to include all combinations and identify missing records
    group_full = group.set_index(['store_id', 'article_id', 'date']).reindex(idx).reset_index()
    
    # Find missing records
    missing = group_full[group_full['inventory'].isna()]
    
    # Append missing records to the final DataFrame
    missing_records = pd.concat([missing_records, missing])

# Display missing records
print("Missing Records:")
print(missing_records)


Total Inventory: 105
Missing Records:
   store_id  article_id       date  inventory
1         1         101 2023-10-02        NaN


In [4]:
df

,store_id,article_id,date,inventory
0,1,101,2023-10-01,10
1,1,101,2023-10-02,15
2,1,102,2023-10-01,5
3,2,101,2023-10-01,20
4,2,102,2023-10-01,25
5,2,102,2023-10-02,30


TypeError: duplicated() got an unexpected keyword argument 'take_last'

In [18]:
# Converting the 'Transaction Creation Timestamp' to datetime for time-based operations
df['Transaction Creation Timestamp'] = pd.to_datetime(df['Transaction Creation Timestamp'], errors='coerce')

# Sorting by article_id, site_id, and transaction creation timestamp to easily find close transactions
df_sorted = df.sort_values(['Generic Article ID', 'SKU', 'Transaction Creation Timestamp'])

# Define the threshold for "close in time" (e.g., within 5 minutes)
time_threshold = pd.Timedelta(minutes=5)

# Find pairs of rows with opposite sign Net Transaction Unit values for the same article, SKU, and close timestamps
to_remove = set()  # Use a set to avoid duplicate entries

for idx, row in df_sorted.iterrows():
    # Skip rows that are already marked for removal
    if idx in to_remove:
        continue
    
    # Find the current article and SKU
    current_article = row['Generic Article ID']
    current_sku = row['SKU']
    current_site = row['Site Banner']
    current_time = row['Transaction Creation Timestamp']
    
    # Check for similar transactions within the time threshold
    similar_transactions = df_sorted[(df_sorted['Generic Article ID'] == current_article) &
                                     (df_sorted['SKU'] == current_sku) &
                                     (df_sorted['Site Banner'] == current_site) &
                                     (df_sorted['Transaction Creation Timestamp'] > current_time) &
                                     (df_sorted['Transaction Creation Timestamp'] <= current_time + time_threshold)]
    
    # Find the opposite sign transaction and remove both
    for similar_idx, similar_row in similar_transactions.iterrows():
        if row['Net Transaction Unit'] + similar_row['Net Transaction Unit'] == 0:
            # Mark both current and similar rows for removal
            to_remove.add(idx)
            to_remove.add(similar_idx)
            break  # No need to check further once a pair is found

# Remove the marked rows
df_filtered = df_sorted.drop(index=to_remove)

# Display the filtered dataframe
df_filtered.head()



,Unnamed: 0,Fiscal Year,Fiscal Week ID,Date,Transaction Date,Transaction Creation Timestamp,Transaction Number,Transaction Line Number,Hybris Ordered Number,Site Banner,...,Colour,Colour ID,Generic Article ID,Size Value,SKU,Channel Group,Gross Transaction Unit,Net Transaction Unit,Net Transaction Value (VAT OUT),Gross Transaction Value
3,131765,2024,14,2024-05-11,2024/05/11 00:00:00,2024-05-11 18:55:21,129762.0,300,NaN,ALDO_CA,...,BLACK,1.0,12638122,NaN,61945374,Stores,1,1,65.0,65.0
0,125901,2024,14,2024-05-11,2024/05/11 00:00:00,2024-05-11 19:10:21,129772.0,400,NaN,ALDO_CA,...,BLACK,1.0,12638122,NaN,61945374,Stores,1,1,65.0,65.0
1,129481,2024,14,2024-05-11,2024/05/11 00:00:00,2024-05-11 19:10:21,129775.0,300,NaN,ALDO_CA,...,BLACK,1.0,12638122,NaN,61945374,Stores,1,1,65.0,65.0
4,131845,2024,14,2024-05-11,2024/05/11 00:00:00,2024-05-11 19:10:21,129775.0,100,NaN,ALDO_CA,...,BLACK,1.0,12638122,NaN,61945374,Stores,0,-1,-65.0,0.0
5,133669,2024,14,2024-05-11,2024/05/11 00:00:00,2024-05-11 20:05:21,217729.0,200,NaN,ALDO_CA,...,BLACK,1.0,12638122,NaN,61945374,Stores,1,1,65.0,65.0


In [20]:
csv_file_path = 'C:\\Users\\hp\\Desktop\\MMA-McGill\\RAship\\ALDO_CA\\03-Ecom\\Additional_trxns.csv'
df_filtered = pd.read_csv(csv_file_path)

# Converting the 'Transaction Creation Timestamp' to datetime for time-based operations
df_filtered['Transaction Creation Timestamp'] = pd.to_datetime(df_filtered['Transaction Creation Timestamp'], errors='coerce')

# Identify transactions with negative Net Transaction Unit
negative_transactions = df_filtered[df_filtered['Net Transaction Unit'] < 0]

# Iterate over negative transactions to find and remove matching positive transactions
for idx, neg_row in negative_transactions.iterrows():
    # Find matching positive transaction
    matching_pos = df_filtered[(df_filtered['Generic Article ID'] == neg_row['Generic Article ID']) &
                               (df_filtered['Site Banner'] == neg_row['Site Banner']) &
                               (df_filtered['Transaction Creation Timestamp'].dt.date == neg_row['Transaction Creation Timestamp'].date()) &
                               (df_filtered['Net Transaction Unit'] == -neg_row['Net Transaction Unit'])]
    
    if not matching_pos.empty:
        # Remove both negative and matching positive transactions
        df_filtered = df_filtered.drop(index=[idx, matching_pos.index[0]])

# Display the updated dataframe
df_filtered.head()

,Unnamed: 0,Fiscal Year,Fiscal Week ID,Date,Transaction Date,Transaction Creation Timestamp,Transaction Number,Transaction Line Number,Hybris Ordered Number,Site Banner,...,Colour,Colour ID,Generic Article ID,Size Value,SKU,Channel Group,Gross Transaction Unit,Net Transaction Unit,Net Transaction Value (VAT OUT),Gross Transaction Value
1,0,2024,14,2024-05-11,2024/05/11 00:00:00,2024-05-11 19:10:21,129772.0,400,NaN,ALDO_CA,...,BLACK,1.0,12638122,NaN,61945374,Stores,1,1,65.0,65.0
2,1,2024,14,2024-05-11,2024/05/11 00:00:00,2024-05-11 19:10:21,129775.0,300,NaN,ALDO_CA,...,BLACK,1.0,12638122,NaN,61945374,Stores,1,1,65.0,65.0
4,5,2024,14,2024-05-11,2024/05/11 00:00:00,2024-05-11 20:05:21,217729.0,200,NaN,ALDO_CA,...,BLACK,1.0,12638122,NaN,61945374,Stores,1,1,65.0,65.0
5,2,2024,14,2024-05-11,2024/05/11 00:00:00,2024-05-11 20:45:21,634586.0,700,NaN,ALDO_CA,...,BLACK,1.0,12638122,NaN,61945374,Stores,1,1,32.5,32.5


In [15]:
df

,Unnamed: 0,Fiscal Year,Fiscal Week ID,Date,Transaction Date,Transaction Creation Timestamp,Transaction Number,Transaction Line Number,Hybris Ordered Number,Site Banner,...,Colour,Colour ID,Generic Article ID,Size Value,SKU,Channel Group,Gross Transaction Unit,Net Transaction Unit,Net Transaction Value (VAT OUT),Gross Transaction Value
0,125901,2024,14,2024-05-11,2024/05/11 00:00:00,2024/05/11 19:10:21,129772.0,400,NaN,ALDO_CA,...,BLACK,1.0,12638122,NaN,61945374,Stores,1,1,65.0,65.0
1,129481,2024,14,2024-05-11,2024/05/11 00:00:00,2024/05/11 19:10:21,129775.0,300,NaN,ALDO_CA,...,BLACK,1.0,12638122,NaN,61945374,Stores,1,1,65.0,65.0
2,130765,2024,14,2024-05-11,2024/05/11 00:00:00,2024/05/11 20:45:21,634586.0,700,NaN,ALDO_CA,...,BLACK,1.0,12638122,NaN,61945374,Stores,1,1,32.5,32.5
3,131765,2024,14,2024-05-11,2024/05/11 00:00:00,2024/05/11 18:55:21,129762.0,300,NaN,ALDO_CA,...,BLACK,1.0,12638122,NaN,61945374,Stores,1,1,65.0,65.0
4,131845,2024,14,2024-05-11,2024/05/11 00:00:00,2024/05/11 19:10:21,129775.0,100,NaN,ALDO_CA,...,BLACK,1.0,12638122,NaN,61945374,Stores,0,-1,-65.0,0.0
5,133669,2024,14,2024-05-11,2024/05/11 00:00:00,2024/05/11 20:05:21,217729.0,200,NaN,ALDO_CA,...,BLACK,1.0,12638122,NaN,61945374,Stores,1,1,65.0,65.0


In [21]:
# Filter transactions where Net Transaction Unit is negative
negative_transactions = df[df['Net Transaction Unit'] < 0]

# Group by Date and filter groups where the number of transactions is greater than 1
filtered_transactions = negative_transactions.groupby('Date').filter(lambda x: len(x) > 1)

# Display the filtered transactions
filtered_transactions

# Fix the issue with groupby count
new_df = trxn_df.groupby(['Site ID', 'Article ID']).size().reset_index(name='Transaction Count')
new_df


# Group by 'Site ID' and 'Article ID' and sum the 'Net Transaction Unit'
grouped_df = df.groupby(['Site ID', 'Article ID'])['Net Transaction Unit'].sum().reset_index()

# Display the grouped dataframe
grouped_df


,Unnamed: 0,Fiscal Year,Fiscal Week ID,Date,Transaction Date,Transaction Creation Timestamp,Transaction Number,Transaction Line Number,Hybris Ordered Number,Site Banner,...,Colour,Colour ID,Generic Article ID,Size Value,SKU,Channel Group,Gross Transaction Unit,Net Transaction Unit,Net Transaction Value (VAT OUT),Gross Transaction Value


In [ ]:
import pandas as pd

# Sample data
data = {
    'Transaction Creation Timestamp': ['2023-10-01 10:00:00', '2023-10-01 11:00:00', '2023-10-01 12:00:00', '2023-10-01 13:00:00'],
    'Article ID': [101, 101, 102, 102],
    'Site ID': [1, 1, 2, 2],
    'Net Transaction Unit': [-5, 5, -10, 10]
}

# Create DataFrame
examp1 = pd.DataFrame(data)

# Converting the 'Transaction Creation Timestamp' to datetime for time-based operations
examp1['Transaction Creation Timestamp'] = pd.to_datetime(examp1['Transaction Creation Timestamp'], errors='coerce')

# Identify transactions with negative Net Transaction Unit
negative_transactions = examp1[examp1['Net Transaction Unit'] < 0]

# Iterate over negative transactions to find and remove matching positive transactions
for idx, neg_row in negative_transactions.iterrows():
    # Find matching positive transaction
    matching_pos = examp1[(examp1['Article ID'] == neg_row['Article ID']) &
                          (examp1['Site ID'] == neg_row['Site ID']) &
                          (examp1['Net Transaction Unit'] == -neg_row['Net Transaction Unit'])]
    
    if not matching_pos.empty:
        # Remove both negative and matching positive transactions
        examp1 = examp1.drop(index=[idx, matching_pos.index[0]])

# Display the updated dataframe
print(examp1)